In [3]:
# !pip install openai
# !git clone https://github.com/Trustworthy-ML-Lab/Label-free-CBM.git
# !pip install ftfy regex tqdm
# !pip install -r requirements.txt
# !pip install pytorchcv
import os
os.chdir('Label-free-CBM')
import json
import openai
import data_utils

"""Uncomment if working with google colab"""
# from google.colab import drive
# drive.mount('/content/MyDrive')


[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Cloning into 'Label-free-CBM'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 100 (delta 25), reused 90 (delta 21), pack-reused 0
Receiving objects: 100% (100/100), 6.51 MiB | 3.58 MiB/s, done.
Resolving deltas: 100% (25/25), done.

[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 9.7 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 kB 9.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing meta

'Uncomment if working with google colab'

In [12]:
"""
Refers to the dataset you wish to generate concepts for...
    For example "cifar10" generates concepts for the following:
        0 airplane
        1 automobile
        2 bird
        3 cat
        4 deer
        5 dog
        6 frog
        7 horse
        8 ship
        9 truck
"""
dataset = "cifar10"
prompt_type = "important"

In [5]:
# replace with personal api key
openai.api_key = open(os.path.join(os.path.expanduser("~"), ".openai_api_key"), "r").read()[:-1]

In [16]:
prompts = {
    "important" : "List the most important features for recognizing something as a \"goldfish\":\n\n-bright orange color\n-a small, round body\n-a long, flowing tail\n-a small mouth\n-orange fins\n\nList the most important features for recognizing something as a \"beerglass\":\n\n-a tall, cylindrical shape\n-clear or translucent color\n-opening at the top\n-a sturdy base\n-a handle\n\nList the most important features for recognizing something as a \"{}\":",
    "superclass" : "Give superclasses for the word \"tench\":\n\n-fish\n-vertebrate\n-animal\n\nGive superclasses for the word \"beer glass\":\n\n-glass\n-container\n-object\n\nGive superclasses for the word \"{}\":",
    "around" : "List the things most commonly seen around a \"tench\":\n\n- a pond\n-fish\n-a net\n-a rod\n-a reel\n-a hook\n-bait\n\nList the things most commonly seen around a \"beer glass\":\n\n- beer\n-a bar\n-a coaster\n-a napkin\n-a straw\n-a lime\n-a person\n\nList the things most commonly seen around a \"{}\":"
}

base_prompt = prompts[prompt_type]

In [17]:
cls_file = data_utils.LABEL_FILES[dataset]
with open(cls_file, "r") as f:
    classes = f.read().split("\n")

In [20]:
feature_dict = {}

for i, label in enumerate(classes):
    feature_dict[label] = set()
    print("\n", i, label)
    for _ in range(2):
        response = openai.completions.create(
              model="text-davinci-002",
              prompt=base_prompt.format(label),
              temperature=0.7,
              max_tokens=500,
              top_p=1,
              frequency_penalty=0,
              presence_penalty=0
            )
        #clean up responses
        features = response.choices[0].text # openai updated version
    #     features = response.choices[0]["text"]
        features = features.split("\n-")
        features = [feat.replace("\n", "") for feat in features]
        features = [feat.strip() for feat in features]
        features = [feat for feat in features if len(feat)>0]
        features = set(features)
        feature_dict[label].update(features)
    feature_dict[label] = sorted(list(feature_dict[label]))


 0 airplane

 1 automobile

 2 bird

 3 cat

 4 deer

 5 dog

 6 frog

 7 horse

 8 ship

 9 truck


In [21]:
json_object = json.dumps(feature_dict, indent=4)
savepath = f"data/concept_sets/gpt3_init/new_concepts/"
if not os.path.exists(savepath):
    os.makedirs(savepath)
with open(f"/Users/jiezy/Desktop/Rice Fall 2023/MedicalCBM/{dataset}_{prompt_type}_concepts.json", "w") as outfile:
    outfile.write(json_object)